In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Masking, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

data = pd.read_csv('combined.csv')
sequences = []
labels = []

grouped = data.groupby(['Gesture', 'Video_ID'])

for (gesture, video_id), video_data in grouped:
    keypoints = video_data.iloc[:, 3:].values
    sequences.append(keypoints)
    labels.append(gesture)

max_seq_len = 204
X_sequences = pad_sequences(sequences, maxlen=max_seq_len, padding='post', truncating='post', value=-4)
y_labels = np.array(labels)

X_train_sequences, X_validation_sequences, y_train_labels, y_validation_labels = train_test_split(
    X_sequences, y_labels, test_size=0.2, stratify=y_labels, random_state=42)

model = Sequential()
model.add(Masking(mask_value=-4, input_shape=(max_seq_len, X_sequences.shape[2])))
model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(64))
model.add(Dropout(0.3))
num_classes = len(np.unique(y_labels))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class EarlyStoppingAtAccuracy(Callback):
    def __init__(self, target_accuracy=0.92):
        super(EarlyStoppingAtAccuracy, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get('accuracy')
        if current_accuracy is not None and current_accuracy >= self.target_accuracy:
            print(f"\nReached {self.target_accuracy * 100:.2f}% accuracy, stopping training.")
            self.model.stop_training = True

early_stopping = EarlyStoppingAtAccuracy(target_accuracy=0.92)

history = model.fit(X_train_sequences, y_train_labels, 
                    epochs=1000, batch_size=64, 
                    validation_data=(X_validation_sequences, y_validation_labels),
                    callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_validation_sequences, y_validation_labels)
print(f'Test accuracy: {accuracy:.2%}')


C:\Users\ankit\AppData\Local\Temp\ipykernel_26480\259185157.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\ankit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 31s 277ms/step - accuracy: 0.0478 - loss: 3.4787 - val_accuracy: 0.1029 - val_loss: 2.9627
Epoch 2/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 31s 305ms/step - accuracy: 0.1128 - loss: 2.9517 - val_accuracy: 0.2059 - val_loss: 2.5384
Epoch 3/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 30s 292ms/step - accuracy: 0.1774 - loss: 2.6123 - val_accuracy: 0.3143 - val_loss: 2.1924
Epoch 4/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 30s 293ms/step - accuracy: 0.2881 - loss: 2.2480 - val_accuracy: 0.4320 - val_loss: 1.8489
Epoch 5/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 31s 300ms/step - accuracy: 0.3653 - loss: 2.0018 - val_accuracy: 0.5196 - val_loss: 1.6497
Epoch 6/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 31s 299ms/step - accuracy: 0.4440 - loss: 1.7601 - val_accuracy: 0.6391 - val_loss: 1.3791
Epoch 7/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 30s 297ms/step - accuracy: 0.5404 - loss: 1.5178 - val_accuracy: 0.6366 - val_loss: 1.2696
Epoch 8/1000
102/102 ━━━━━━━━━━━━━━━━━━━━ 29s 282ms/step - accuracy: 

In [ ]:
model.save('gesture_recognition_model.keras')


In [ ]:
model.save('gesture_recognition_model.h5')
